Note: Colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint.

clone and cd into repo, nshepperd's fork https://github.com/nshepperd/gpt-2

In [ ]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 375, done.
remote: Total 375 (delta 0), reused 0 (delta 0), pack-reused 375
Receiving objects: 100% (375/375), 4.42 MiB | 1.35 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [ ]:
cd gpt-2

/content/gpt-2


Install requirements

In [ ]:
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15
!pip install 'tensorflow-estimator<1.15.0rc0,>=1.14.0rc0' --force-reinstall
!pip3 install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/de/62/2ee9cd74c9fa2fa450877847ba560b260f5d0fb70ee0595203082dafcc9d/tensorflow_estimator-1.15.1-py2.py3-none-any.whl
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
ERROR: tensorflow-gpu 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
ERROR: Could not open requirements file: [Errno 2]

Mount drive to access google drive for saving and accessing checkpoints later. Have to log in to your google account

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Download the model data

In [ ]:
!python3 download_model.py 117M

In [ ]:
!python3 download_model.py 345M

encoding

In [ ]:
!export PYTHONIOENCODING=UTF-8

Fetch checkpoints if you have them saved in google drive

In [ ]:
!cp -r /content/drive/My\ Drive/checkpoint/ /content/gpt-2/ 


Let's get our train on! In this case the file is A Tale of Two Cities (Charles Dickens) from Project Gutenberg. To change the dataset GPT-2 models will fine-tune on, change this URL to another .txt file, and change corresponding part of the next cell. Note that you can use small datasets if you want but you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.


Start training, add --model_name '345M' to use 345 model

In [ ]:
!PYTHONPATH=src ./train.py --dataset /content/gpt-2/all.txt --model_name '345M'

Save our checkpoints to start training again later

In [ ]:
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/

Load your trained model for use in sampling below (117M or 345M)

In [ ]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

^C


In [ ]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"

In [ ]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"

To check flag descriptions, use:

In [ ]:
!python3 src/interactive_conditional_samples.py -- --help


NAME
    interactive_conditional_samples.py - Interactively run the model :model_name=117M : String, which model to use :seed=None : Integer seed for random number generators, fix seed to reproduce results :nsamples=1 : Number of samples to return total :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples. :length=None : Number of tokens in generated text, if None (default), is determined by model hyperparameters :temperature=1 : Float value controlling randomness in boltzmann distribution. Lower temperature results in less random completions. As the temperature approaches zero, the model will become deterministic and repetitive. Higher temperature results in more random completions. :top_k=0 : Integer value controlling diversity. 1 means only 1 word is considered for each step (token), resulting in deterministic completions, while 40 means 40 words are considered at each step. 0 (default) is a special setting meaning no restrictions. 40 generally is a 

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"

In [ ]:
!python3 src/generate_unconditional_samples.py --model_name "345M" | tee /tmp/samples

To check flag descriptions, use:

In [ ]:
!python3 src/generate_unconditional_samples.py -- --help